<a href="https://colab.research.google.com/github/xborrat/NEFRoHack/blob/main/notebooks/intro-to-sql-mimic-iv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to SQL starring MIMIC!

The aim of this tutorial is to introduce Structured Query Language (SQL) through a publicly available medical database: the MIMIC-IV Clinical Database Demo.

The general learning objectives for this tutorial are for you to have:

- an understanding of relational databases
- familiarity with the MIMIC-IV database
- ability to select data from a database using Structured Query Language (SQL)
- ability to reuse code in the MIMIC Code Repository
- ability to use SQL aggregate and window functions

In addition, specific to MIMIC, you will:

* Understand what `subject_id`, `hadm_id`, and `icustay_id` represent

Requirements:

* Access to the MIMIC demo datasets on BigQuery.
    * You need a Google project which you can bill usage to. When you first create a Google account, you are given $300 credit which is more than enough.
    * Pin the `physionet-data` project on the BigQuery page. [Pinning is described on the BigQuery Web UI documentation from Google](https://cloud.google.com/bigquery/docs/bigquery-web-ui).
    * If you would like to use the full version of MIMIC rather than the demo, you'll need to follow the [instructions for being granted access to the MIMIC BigQuery projects](https://mimic.mit.edu/docs/gettingstarted/cloud/).


Our first few cells will be a bit of setup.

In [ ]:
# Import libraries
from datetime import timedelta
import os
import sqlite3
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# below imports are used for pretty pandas dataframes and plots
from IPython.display import display, HTML
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 22})

We need to authenticate this notebook with Google Cloud Platform (GCP) in order to query MIMIC on BigQuery.

In [ ]:
# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# You only need to authenticate once per session.
auth.authenticate_user()

Now we will set the project name. **You will need to change the project_id in the below cell to your own project.**

In [ ]:
# Set up environment variables
project_id = 'lcp-internal'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
    return pd.io.gbq.read_gbq(
        query,
        project_id=project_id,
        dialect="standard"
    )

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimic'

if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project at the top of this cell.')

# test it works
df = run_query("""
SELECT *
FROM physionet-data.mimiciv_hosp.patients
WHERE subject_id = 10012853
""")
assert df.shape[0] >= 1, 'unable to query MIMIC!'
display(df)

If the above raises an error, you'll need to double check you've set your project correctly, and ensure that you have [requested access to MIMIC-IV on Google BigQuery via the PhysioNet project page](https://mimic-iv.mit.edu/docs/access/cloud/#accessing-mimic-iv-on-the-cloud).


## Introduction to MIMIC-IV

MIMIC-IV is a publicly available dataset developed by the MIT Lab for Computational Physiology, comprising deidentified health data associated with >60,000 hospital stays.

The dataset is provided as a collection of comma-separated value (CSV) files, which we have loaded into SQLite above. A list of tables is provided on the MIMIC website: https://mimic.mit.edu/docs/iv/modules/

We have highlighted some of the key tables below:

- *patients*: the set of patients included in the database, each identified by a unique `subject_id`.
- *admissions*: the set of hospital admissions, each identified by a unique `hadm_id`.
- *icustays*: the set of ICU stays, each identified by a unique `stay_id`.

### Example questions

1. Where would I find a patient's date of birth?
2. Where would I find a patient's hospital admission time?

## Example solutions

1. The patient's date of birth can be found in the *patients* table, as detailed in the documentation: https://mimic.mit.edu/docs/iv/modules/hosp/patients/
2. The patient's hospital admission time can be found in the *admissions* table, which tracks hospital admission information, as detailed here: https://mimic.mit.edu/docs/iv/modules/hosp/admissions/



## Relational databases

Relational databases can be thought of as a collection of tables which are linked together by shared keys. Organizing data across tables can help to maintain data integrity and enable faster analysis and more efficient storage.

### Motivation: why would we want a relational database?

Imagine trying to store data about a person: their name, age, and height. We can easily save this data in a CSV:

    "Name", "Age", "Height"
    "Penny",     30,     182

Now what if we measure Penny's heart rate every hour for four hours at 8:00am, 9:00am, 10:00am, and 11:00 am.
How should we store this data? The naive approach would be to simply concatenate the information we have all in one file:

    "Name", "Age", "Height",  "Time", "Heart rate"
    "Penny",     30,      182,  "8:00",          65
    "Penny",     30,      182,  "9:00",          71
    "Penny",     30,      182, "10:00",          72
    "Penny",     30,      182, "11:00",          68

This works, but it feels very inefficient. We have repeated her name ("Penny"), her age (30), and her height (182) every time we get a heart rate measurement. The immediate solution is to not store both of these in the same file: we make one file for Penny's demographics (age, height), and we make another file for heart rate measurements. Then, we make sure that her name is the same in both, so that we know that both sets of data relate to Penny. We've created a relational database. Since her name is what links the two tables together, we would call the name column a "key".

### Database terminology

- "Database schema": The model that defines the structure and relationships of the tables.
- "Database query": Data is extracted from relational databases using structured "queries".
- "Primary key": A primary key is a field that uniquely identifies each row in a table.
- "Foreign key": A foreign key is a field that refers to a primary key in another table.
- "Normalisation": The concept of structuring a database in a way that reduces data repetition and improves data integrity, usually by requiring one or more tables to be joined.
- "Denormalisation": The concept of structuring a database to improve readability, sometimes at the expense of data repetition and data integrity.
- "Data type": A term used to describe the behaviour of data and the possible values that it can hold (for example, integer, text, and date are all data types in PostgreSQL).

Giving a simple example of a hospital database with four tables, it might comprise of: Table 1, a list of all patients; Table 2, a log of hospital admissions; Table 3, a list of vital sign measurements; Table 4, a dictionary of vital sign codes and associated labels.

The patients table lists unique patients. The admissions table lists unique hospital admissions. The chartevents table lists charted events such as heart rate measurements. The `d_items` table is a dictionary that lists `itemid`s and associated labels, as shown in the example query. pk is primary key. fk is foreign key.



## What is Structured Query Language (SQL)?

Structured Query Language (SQL) is a programming language used to manage relational databases.
An SQL query has the following format:

```
SELECT [columns]
FROM [table_name];
```

The simplest SQL query select a set of `[columns]` from a specific location, the `[table_name]`. Here's a real example you can run using the SQLite database loaded in. We know `subject_id` is in the `patients` table, so let's try to query for them.

In [ ]:
df = run_query(f"""
SELECT subject_id
FROM physionet-data.mimiciv_hosp.patients
""")
display(df)

Sometimes it can be useful to select all the columns. Although SQL rarely allows wildcards, the `*` can be used to select all possible columns in a table.


In [ ]:
df = run_query(f"""
SELECT *
FROM physionet-data.mimiciv_hosp.patients
""")
display(df)

Try it yourself! Update the below query to select only the `subject_id` and `gender` columns.

In [ ]:
df = run_query(f"""
SELECT *
FROM physionet-data.mimiciv_hosp.patients
""")
display(df)


### `WHERE` keyword

Often you will want to select a subset of the data which satisfy some set of conditions.
For example, you may want to select only female subjects from the database.
This is easily accomplished with the `WHERE` keyword. The framework of our query becomes:

```
SELECT [columns]
FROM [table_name]
WHERE [conditions];
```

We can easily select all the `subject_id` corresponding to female subjects as follows:


In [ ]:
df = run_query(f"""
SELECT subject_id
FROM physionet-data.mimiciv_hosp.patients
WHERE gender = 'F';
""")
display(df)

`WHERE` clauses are used to make a query return rows meeting only our specified criteria (our previous query, for example, returning only female patients). The simplest criteria is equality, `WHERE gender = 'F'`. Note that in this situation we specify a string, but this syntax will work for numbers as well. For example, we could select all the data for a single subject:


In [ ]:
df = run_query(f"""
SELECT *
FROM physionet-data.mimiciv_hosp.patients
WHERE subject_id = 10008454;
""")
display(df)

`WHERE` clauses can be combined with standard logical operators `AND`/`OR`:


In [ ]:
df = run_query(f"""
SELECT *
FROM physionet-data.mimiciv_hosp.patients
WHERE subject_id = 10008454
OR subject_id = 10010471
OR subject_id = 999;
""")
display(df)

A useful shorthand for `OR` statements on the same column is the `IN` condition:


In [ ]:
df = run_query(f"""
SELECT *
FROM physionet-data.mimiciv_hosp.patients
WHERE subject_id IN (10008454, 10010471, 999);
""")
display(df)

Above, we can see we have returned two rows, as there were two matches. While we included `999` in the `IN` clause, no `subject_id` had a value of `999`.

We can also use the "less than" (`<`), "less than or equal to" `<=`, "greater than" (`>`), or "greater than or equal to" `>=` operators:


In [ ]:
df = run_query(f"""
SELECT *
FROM physionet-data.mimiciv_hosp.patients
WHERE subject_id >= 10008000
AND subject_id <= 10012000;
""")
display(df)

SQL also offers shorthand for `>=` and `<=` combinations with the `BETWEEN` condition:


In [ ]:
df = run_query(f"""
SELECT *
FROM physionet-data.mimiciv_hosp.patients
WHERE subject_id BETWEEN 10008000 AND 10012000;
""")
display(df)

Note the `BETWEEN` operator is inclusive. Verify for yourself that the above two queries give the same result.

When working with text data, we'll often want to search for partial string matches rather than exact matches. This can be accomplished with the `LIKE` keyword:


In [ ]:
df = run_query(f"""
SELECT *
FROM physionet-data.mimiciv_icu.icustays
WHERE first_careunit LIKE '%ICU%';
""")
display(df)

Note the use of the wildcard character `%`, which matches any text. Note the placement of two wildcard operators.

* `LIKE 'ICU%'` - match text that *starts with ICU*
* `LIKE '%ICU'` - match text that *ends with ICU*
* `LIKE '%ICU%'` - match text that *contains the letters ICU*


### ORDER BY keyword

The `ORDER BY` keyword is relatively straightforward: it will sort the data in the order you specify.

```
SELECT [columns]
FROM [table_name]
WHERE [conditions]
ORDER BY [columns];
```

The below query orders the results by the patient `anchor_age`


In [ ]:
df = run_query(f"""
SELECT subject_id, anchor_age
FROM physionet-data.mimiciv_hosp.patients
ORDER BY anchor_age
""")
display(df)

Note that the `WHERE` clause is optional, and in the above query we have omitted it. However, we must respect the order of the keywords - and if we use the `WHERE` keyword it must appear after the `FROM` keyword and before the `ORDER BY` keyword.

For simplicity, we can also replace the column names in the `ORDER BY` clause with the numeric column index (1-index). The following returns the same result as the above query.

In [ ]:
df = run_query(f"""
SELECT subject_id, anchor_age
FROM physionet-data.mimiciv_hosp.patients
ORDER BY 2
""")
display(df)


### JOIN keyword

Often we need information coming from multiple tables. This can be achieved using the SQL `JOIN` keyword. There are several types of join, including `INNER JOIN`, `LEFT JOIN`, and `RIGHT JOIN`. It is important to understand the difference between these joins because their usage can significantly impact query results. Detailed guidance on joins is widely available on the web.

Using the `INNER JOIN` keyword, let's select a list of patients from the *patients* table along with dates of birth, and join to the *admissions* table to get the admission time for each hospital admission. We use the `INNER JOIN` to indicate that two or more tables should be combined based on a common attribute, which in our case is `subject_id`:


In [ ]:
df = run_query(f"""
SELECT p.subject_id, a.hadm_id, a.admittime
FROM physionet-data.mimiciv_hosp.patients p
INNER JOIN physionet-data.mimiciv_hosp.admissions a
ON p.subject_id = a.subject_id
ORDER BY p.subject_id, a.admittime
""")
display(df)

Note that an `INNER JOIN` will only return rows where subject_id appears in both the patients table and the admissions table.

A `LEFT JOIN` will include every row on the left, but only matching rows on the right. So we join *from the left*.

A `RIGHT JOIN` will include every row on the right, but only matching rows on the left. So we join *from the right*.

Don't worry if you are bad with your lefts and rights. Practically speaking, you only need `INNER JOIN` and `LEFT JOIN`. If you really insist, you can use `RIGHT JOIN`, but it's unconventional.

### ROUND and other functions

Sometimes we will want to perform operations on columns. For example, if we are only interested in length of stay (`los`) to the nearest day, we can use the `round` function:


In [ ]:
df = run_query(f"""
SELECT hadm_id, round(los)
FROM physionet-data.mimiciv_icu.icustays;
""")
display(df)

Note that the column name ends up being `round(los)`. We can specify the column name using the `AS` keyword:


In [ ]:
df = run_query(f"""
SELECT hadm_id, round(los) AS los_integer_days
FROM physionet-data.mimiciv_icu.icustays;
""")
display(df)

There are a large number of operations available in SQL.  Since we are using SQLite, we would look at the [SQLite core functions page](https://www.sqlite.org/lang_corefunc.html), which includes [round(X)](https://www.sqlite.org/lang_corefunc.html#round).




### CASE statement for if/else logic

The `CASE` statement is used to handle if/else logic. For example, using the *patients* table you may want to group patients by age.


In [ ]:
df = run_query(f"""
SELECT subject_id, anchor_age,
    CASE WHEN anchor_age < 20 THEN 'youth'
         WHEN anchor_age < 40 THEN 'young adult'
         WHEN anchor_age < 60 THEN 'middle adult'
         WHEN anchor_age >= 60 THEN 'older adult'
         ELSE NULL END AS age_group
FROM physionet-data.mimiciv_hosp.patients;
""")
display(df)

### Aggregate functions

We are often interested in finding an aggregate value across multiple rows, such as the number of patients meeting a condition, an average heart rate, or a maximum blood pressure. We can do this using aggregate functions, such as `COUNT()`, `MAX()`, `SUM()`, and `AVG()`.

Count the number of rows in the *icustays* table with the `COUNT()` function:


In [ ]:
df = run_query(f"""
SELECT count(*)
FROM physionet-data.mimiciv_icu.icustays;
""")
display(df)


Find the maximum length of stay in the *icustays* table with the `MAX()` function:


In [ ]:
df = run_query(f"""
SELECT MAX(los)
FROM physionet-data.mimiciv_icu.icustays;
""")
display(df)

Aggregate are often combined with a `GROUP BY` clause, so that the aggregate function is applied to specific groups. For example, we can find the maximum length of stay, grouped for each patient:

In [ ]:
df = run_query(f"""
SELECT subject_id, MAX(los)
FROM physionet-data.mimiciv_icu.icustays
GROUP BY subject_id;
""")
display(df)

Notice how the above now returns 100 rows, whereas we originally had 140 ICU stays.

We may want to add a condition based on our new aggregate column. The `WHERE` clause won't filter on an aggregate column, so instead we use the `HAVING` keyword. For example, we can find the maximum length of stay, grouped for each patient, returning only patients whose maximum stay is less than 10 days:

In [ ]:
df = run_query(f"""
SELECT subject_id, MAX(los)
FROM physionet-data.mimiciv_icu.icustays
GROUP BY subject_id
HAVING MAX(los) <= 10;
""")
display(df)

This returns fewer rows than the above, as we've filtered out those with long stays (>= 10 days).

### Window functions

Sometimes an aggregate function isn't quite what we need. For example, we might want to create a column that lists the order of admissions to the ICU for each patient. In this case we do not want to group all of the rows with the same `subject_id` into a single row, so a simple a aggregate function is insufficient. Instead, we want to return multiple rows for each `subject_id`, with the order of admission computed over a `subject_id` window. For example, let's find the order of admission to the ICU for each patient using the `RANK()` window function:

In [ ]:
df = run_query(f"""
SELECT subject_id, stay_id, intime,
    RANK() OVER (PARTITION BY subject_id ORDER BY intime)
FROM physionet-data.mimiciv_icu.icustays;
""")
display(df)

### Temporary tables

It is sometimes helpful to create temporary views or tables to break a large query into smaller, more manageable chunks. It can also be useful to *use* the output of a query. For example, in the above we ranked patients by the sequence of their ICU stays. We may want to now filter to first ICU stay.

There are several approaches that can be used to create temporary tables. The simplest method uses the `WITH` keyword. For example, we'll create a temporary view called `icustayorder` using the previous query, and then select all of its columns:


In [ ]:
df = run_query(f"""
WITH icustayorder AS
(
SELECT subject_id, stay_id, intime,
  RANK() OVER (PARTITION BY subject_id ORDER BY intime) AS rank
FROM physionet-data.mimiciv_icu.icustays
)
SELECT *
FROM icustayorder
WHERE rank = 1;
""")
display(df)

Note we also needed to give the column a name, which we did using the `AS` keyword (`AS rank`).


# Multiple temporary views

Using the `WITH` statement, you can have more than one inline view. The `services` table contains information about what type of care a patient is receiving in the hospital.

In [ ]:
df = run_query(f"""
SELECT subject_id, hadm_id, transfertime, prev_service, curr_service
FROM physionet-data.mimiciv_hosp.services;
""")
display(df)

Note that the `services` table doesn't have `stay_id`, but we can join to it using `hadm_id`.

In [ ]:
df = run_query(f"""
WITH serv as (
  SELECT subject_id, hadm_id, transfertime, prev_service, curr_service
  FROM physionet-data.mimiciv_hosp.services
)
, icu as
(
  SELECT subject_id, hadm_id, stay_id, intime, outtime
  FROM physionet-data.mimiciv_icu.icustays
)
SELECT icu.subject_id, icu.hadm_id, icu.stay_id, icu.intime, icu.outtime
, serv.transfertime, serv.prev_service, serv.curr_service
FROM icu
INNER JOIN serv
ON icu.hadm_id = serv.hadm_id
""")
display(df)

However, something subtle has happened in this join. Let's see how many rows of data are returned by the above query. We can do this using the aggregate operator `COUNT()`:


In [ ]:
df = run_query(f"""
WITH serv as (
  SELECT subject_id, hadm_id, transfertime, prev_service, curr_service
  FROM physionet-data.mimiciv_hosp.services
)
, icu as
(
  SELECT subject_id, hadm_id, stay_id, intime, outtime
  FROM physionet-data.mimiciv_icu.icustays
)
SELECT COUNT(*)
FROM icu
INNER JOIN serv
ON icu.hadm_id = serv.hadm_id
""")
display(df)

We have replaced all the column names with `COUNT(*)` - which means "count all the rows". What result do you get? Let's compare it to the original *icustays* table:

In [ ]:
df = run_query(f"""
SELECT count(*)
FROM physionet-data.mimiciv_icu.icustays;
""")
display(df)

So what happened?

Intuitively, each hospital admission (`hadm_id`) can have multiple service types (i.e. the patient transferred from surgical to medical), and can have multiple ICU stays (i.e. the patient was readmitted to the ICU). As a result, the first query duplicates rows by matching every service the patient was under to every ICU stay the patient had, regardless of whether they match. This is because we are only joining on `hadm_id`, so the only constraint is that the two events occurred in the same hospitalization. Of course, we do not want this to happen, since a patient is only ever on one service at a time. More technically, the first query joined two tables on non-unique keys: there may be multiple `hadm_id` with the same value in the *services* table, and there may be multiple `hadm_id` with the same value in the *admissions* table. For example, if the *services* table has `hadm_id = 100001` repeated N times, and the *admissions* table has `hadm_id = 100001` repeated M times, then joining these two on `hadm_id` will result in a table with NxM rows: one for every pair. With MIMIC, it is generally very bad practice to join two tables on non-unique columns: at least one of the tables should have unique values for the column, otherwise you end up with duplicate rows and the query results can be confusing.


We can fix this by bringing in the `RANK` we worked with earlier, and only join against the *first* service. The general principle here is to **ensure each side of a join is unique**.

In [ ]:
df = run_query(f"""
WITH serv as (
  SELECT subject_id, hadm_id, transfertime, prev_service, curr_service,
    RANK() OVER (PARTITION BY hadm_id ORDER BY transfertime) as rank
  FROM physionet-data.mimiciv_hosp.services
)
, icu as
(
  SELECT subject_id, hadm_id, stay_id, intime, outtime
  FROM physionet-data.mimiciv_icu.icustays
)
SELECT COUNT(*)
FROM icu
INNER JOIN serv
ON icu.hadm_id = serv.hadm_id
AND serv.rank = 1;
""")
display(df)

## Exercise questions

We will be examining data for a single individual `subject_id`.


In [ ]:
subject_id = 10000032

You will f-string syntax frequently as this is used to insert the above `subject_id` into queries. Here's an example:

In [ ]:
query = f"Curly brackets are used to insert variables: {subject_id}."
print(query)

You can [read more about f-strings here](https://docs.python.org/3/tutorial/inputoutput.html).

# Questions

Question 1: Run the below query. How many rows are returned?

In [ ]:
df = run_query(f"""
SELECT *
FROM physionet-data.mimiciv_hosp.patients
""")
display(df)

Answer 1:

Question 2: Write a query which counts the number of rows.


In [ ]:
# Answer 2
df = run_query(f"""

""")
display(df)

Question 3: Write a query which calculates the average of the `valuenum` column.

In [ ]:
# Answer 3
df = run_query(f"""

""")
display(df)

Question 4: Rewrite the above query to use the `value` column. It won't run. What is the error message you receive? What is the reason why this query won't run?

In [ ]:
# Answer 4 - rewrite the query and demonstrate the error
df = run_query(f"""

""")
display(df)

Answer 4 (explain the error):

Question 5: Write a query which identifies the *label* for the given `itemid`. What is the *label*?

In [ ]:
# Answer 5
df = run_query(f"""

""")
display(df)

## Vital signs

Question 6: Run the below query. What is `ce`? Where is it defined? Why is it used?

In [ ]:
df = run_query(f"""
select
  ce.subject_id
  , ce.charttime
  , ROUND(
      AVG(
        CASE WHEN itemid IN (223761) AND valuenum > 70 AND valuenum < 120 THEN (valuenum-32)/1.8
             WHEN itemid IN (223762) AND valuenum > 10 AND valuenum < 50  THEN valuenum
        ELSE null END
      )
    , 2) as merged_value
  , MAX(CASE WHEN itemid = 224642 THEN value ELSE NULL END) AS merged_site
FROM physionet-data.mimiciv_icu.chartevents ce
where ce.subject_id = {subject_id}
AND ce.itemid IN (223761, 223762, 224642)
GROUP BY ce.subject_id, ce.charttime
""")
display(df)

Answer 6:


Question 7: Explain what the CASE statement in the above query is doing.

Answer 7:


Question 8: Copy the above query into the next cell and remove the `charttime` column from the SELECT and GROUP BY statements. Re-run the query. How many rows are returned?


In [ ]:
df = run_query(f"""

""")
display(df)


Answer 8:

Question 9: Did we end up with a different number of rows in Question 8 versus when we originally ran the query (Question 6)? Why or why not?

Answer 9:

Question 10 (bonus): What would change if we used `MIN()` instead of `MAX()` in the above query? Explain your reasoning.

Answer 10:

## Extracting Height

The following query extracts the height of our subject as documented in *chartevents*.

In [ ]:
df = run_query(f"""
-- prep height
WITH ht_in AS
(
  SELECT
    c.subject_id, c.stay_id, c.charttime
    -- Ensure that all heights are in centimeters
    , ROUND(c.valuenum * 2.54, 2) AS height
    , c.valuenum as height_orig
  FROM physionet-data.mimiciv_icu.chartevents c
  WHERE c.valuenum IS NOT NULL
  -- Height (measured in inches)
  AND c.itemid = 226707
  AND c.subject_id = {subject_id}
)
, ht_cm AS
(
  SELECT
    c.subject_id, c.stay_id, c.charttime
    -- Ensure that all heights are in centimeters
    , ROUND(c.valuenum, 2) AS height
  FROM physionet-data.mimiciv_icu.chartevents c
  WHERE c.valuenum IS NOT NULL
  -- Height cm
  AND c.itemid = 226730
  AND c.subject_id = {subject_id}
)
-- merge cm/height, only take 1 value per charted row
, ht_stg0 AS
(
  SELECT
  COALESCE(h1.subject_id, h2.subject_id) as subject_id
  , COALESCE(h1.stay_id, h2.stay_id) AS stay_id
  , COALESCE(h1.charttime, h2.charttime) AS charttime
  , COALESCE(h1.height, h2.height) as height
  FROM ht_cm h1
  LEFT JOIN ht_in h2
    ON h1.subject_id = h2.subject_id
    AND h1.charttime = h2.charttime
  UNION
  SELECT
  COALESCE(h1.subject_id, h2.subject_id) as subject_id
  , COALESCE(h1.stay_id, h2.stay_id) AS stay_id
  , COALESCE(h1.charttime, h2.charttime) AS charttime
  , COALESCE(h1.height, h2.height) as height
  FROM ht_in h1
  LEFT JOIN ht_cm h2
    ON h1.subject_id = h2.subject_id
    AND h1.charttime = h2.charttime
  WHERE h2.subject_id IS NULL
)
SELECT subject_id, stay_id, charttime, height
FROM ht_stg0
WHERE height IS NOT NULL
-- filter out bad heights
AND height > 120 AND height < 230;
""")
display(df)

Question 11: What is the purpose of the `COALESCE` statements?

Answer 11:

## Writing SQL

Question 12: Write a query which lists all hospital admissions for the patient; specifically their `hadm_id`, `admittime`, and `dischtime`.


In [ ]:
df = run_query(f"""

""")
display(df)


Question 13: Write a query which extracts the lowest and highest heart rate for the given `subject_id`.


In [ ]:
df = run_query(f"""

""")
display(df)


Question 14:  Write a query to extract the *first* heart rate for the given `subject_id`.


In [ ]:
df = run_query(f"""

""")
display(df)


Question 15: Write a query which returns all the INR values for the given `subject_id`. INR is a lab value routinely measured for critically ill patients.


In [ ]:
df = run_query(f"""

""")
display(df)


Question 16: Write a query to extract all the medications prescribed to the given `subject_id`, and a count of how many times they were prescribed.

In [ ]:
df = run_query(f"""

""")
display(df)